In [ ]:
import pip
# pip.main(['install', '{insert_pckg_here}'])
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#Exploratory Data Analysis
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

## Loading Training/Testing Data:

In [ ]:
train_url = 'https://raw.githubusercontent.com/mturner49/pylovers-final-project/dev/data/train.csv'
test_url = 'https://raw.githubusercontent.com/mturner49/pylovers-final-project/dev/data/test.csv'

train_df = pd.read_csv(train_url, low_memory=False, error_bad_lines=False, index_col='Id')
test_df = pd.read_csv(test_url, low_memory=False, error_bad_lines=False, index_col='Id')

train_df.head()

In [ ]:
# This is for checking datatypes
train_df.info()

## Data Preprocessing:

In [ ]:
# create df that consists of columns and the number of missing values for each 
data = train_df.isnull().sum().sort_values(ascending=False)
missing_df = pd.DataFrame(data=data, columns=['missing_cnt'])

# add column and fill it with the percentage of those missing values
missing_df['percent_missing'] = missing_df.missing_cnt.apply(lambda x : '{:.2f}'.format(x/train_df.shape[0] * 100)) 
missing_df = missing_df[missing_df.missing_cnt > 0]
missing_df

In [ ]:
# drop the columns where majority of their values are missing
train_df = train_df.drop(['PoolQC', 'MiscFeature', 'Fence', 'FireplaceQu'], axis = 1)
train_df.head()

In [ ]:
# Vida
# I think we should drop 'Alley' as well! with Just 91 non-null values, like 93.77% null!
train_df = train_df.drop(['Alley'], axis = 1)
train_df.head()

In [ ]:
# All 'GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageQual','GarageCond' columns has 1379 not null values!
# So, I tested the theory that exactly same rows has null for these columns!
# All 81 do not have Garage! So, we can put zero instead!
train_df.loc[:,['GarageCars','GarageArea','GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageQual','GarageCond']][train_df['GarageType'].isnull()]


In [ ]:
# All 81 do not have Garage! So, we can put zero instead!
train_df.update(train_df[['GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageQual','GarageCond']].fillna(0))

In [ ]:
# Same for Basement: 'BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1'
train_df.loc[:,['TotalBsmtSF','BsmtUnfSF','BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1']][train_df['BsmtCond'].isnull()]

In [ ]:
# All 37 do not have Garage! So, we can put zero instead!
train_df.update(train_df[['BsmtQual','BsmtCond','BsmtFinType1']].fillna(0))

In [ ]:
# Same for Basement: 'BsmtFinType2','BsmtExposure'
train_df.loc[:,['TotalBsmtSF','BsmtUnfSF','BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1']][train_df['BsmtExposure'].isnull()]

In [ ]:
train_df.at[949,'BsmtExposure']='No'

In [ ]:
train_df.loc[:,['TotalBsmtSF','BsmtUnfSF','BsmtFinSF2','BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1']][train_df['BsmtFinType2'].isnull()]

In [ ]:
# I'm going with 'Average Living Quarters'
train_df.at[333,'BsmtFinType2']='ALQ'

In [ ]:
train_df.update(train_df[['BsmtFinType2','BsmtExposure']].fillna(0))

In [ ]:
# Vida: let's keep the dummies for after Exploratory Data Analysis!
# train_df1 = pd.get_dummies(data=train_df , columns=['MSZoning','Street', 'Alley', 'LotShape', 'LandContour',
#                                                    'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
#                                                    'BldgType', 'HouseStyle'])
# train_df1 = pd.get_dummies(data=train_df , columns=['MSZoning','Street', 'LotShape', 'LandContour',
#                                                    'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
#                                                    'BldgType', 'HouseStyle'])
# train_df1.head()

In [ ]:
# creating a generic dictionary to hold numercial values to represent categorical values
# for quality related columns (ExterQual, BsmtQual)
# quality_ratings = {
#  'NA':0,
#  'Po':1, 
#  'Fa':2, 
#  'TA':3, 
#  'Gd':4, 
#  'Ex':5
# }

# I'm not sure about this trick! but I saw someone metioned the Mean Absolute Percentage Error (MAPE)
# can be good accuracy for just not negative and non zero features!!
# So, to stay in safe side I suggest we start from 1!
quality_ratings = {
 'NA':1,
 'Po':2, 
 'Fa':3, 
 'TA':4, 
 'Gd':5, 
 'Ex':6
}

In [ ]:
# Converting category labels to numerical values for ExterQual column
train_df['ExterQual_Num'] = train_df.ExterQual.map(quality_ratings)

In [ ]:
# Converting category labels to numerical values for BsmtQual column
train_df['BsmtQual_Num'] = train_df.BsmtQual.map(quality_ratings)

In [ ]:
# Converting category labels to numerical values for HeatingQC column
train_df['HeatingQC_Num'] = train_df.HeatingQC.map(quality_ratings)

In [ ]:
# Converting category labels to numerical values for KitchenQual column
train_df['KitchenQual_Num'] = train_df.KitchenQual.map(quality_ratings)

In [ ]:
# Converting category labels to numerical values for GarageQual column
train_df['GarageQual_Num'] = train_df.GarageQual.map(quality_ratings)

In [ ]:
# Converting category labels to numerical values for SaleCondition column
# train_df['SaleCondition_Num'] = train_df.SaleCondition.map(
# {'Abnorml':1, 
#  'AdjLand':2, 
#  'Alloca':3, 
#  'Family':4, 
#  'Normal':5,
#  'Partial':6})

# trying to make the weight meaningful based of some advise from a realstate agent freind!
train_df['SaleCondition_Num'] = train_df.SaleCondition.map(
{'Normal':6,
 'Alloca':5,
 'AdjLand':4,
 'Family':3,
 'Partial':2,
 'Abnorml':1
})

In [ ]:
# drop the columns where majority of their values are missing
train_df = train_df.drop(['ExterQual', 'BsmtQual', 'HeatingQC', 'KitchenQual','GarageQual','SaleCondition'], axis = 1)
train_df.head()

In [ ]:
# create df that consists of columns and the number of missing values for each 
data = train_df.isnull().sum().sort_values(ascending=False)
missing_df = pd.DataFrame(data=data, columns=['missing_cnt'])

# add column and fill it with the percentage of those missing values
missing_df['percent_missing'] = missing_df.missing_cnt.apply(lambda x : '{:.2f}'.format(x/train_df.shape[0] * 100)) 
missing_df = missing_df[missing_df.missing_cnt > 0]
missing_df

In [ ]:
train_df.update(train_df[['Electrical']].fillna('SBrkr'))

In [ ]:
#I'm not professional but I think there is no Masonry veneer type for the Vinyl and Cement!
train_df.at[530,'MasVnrType']='Stone'

# train_df['Set_of_Numbers'] = train_df['Set_of_Numbers'].fillna(0)
train_df.update(train_df[['MasVnrArea']].fillna(0))
train_df.update(train_df[['MasVnrType']].fillna('None'))

## Missing Data Imputation

The rest of missing data is numeric. So, I prefer to replace by mean of same column!

In [ ]:
train_df['LotFrontage'] = train_df['LotFrontage'].fillna((train_df['LotFrontage'].mean()))

In [ ]:
train_df['BsmtQual_Num'] = train_df['BsmtQual_Num'].fillna((train_df['BsmtQual_Num'].mean()))

In [ ]:
train_df['GarageQual_Num'] = train_df['GarageQual_Num'].fillna((train_df['GarageQual_Num'].mean()))

In [ ]:
# Vida: let's speak about this part in our meeting! 
# Vida: I handled all null values one by one!

# NA is a valid value and is some cases not equivalent to NaN and should not be converted to 0
# replace null values in df w/ values that had most counts for each column
train_df1 = train_df1.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' \
                          else x.fillna(train_df.columns.value_counts().idxmax()))
test_df = test_df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' \
                          else x.fillna(train_df.columns.value_counts().idxmax()))

In [ ]:
# Vida: I couldn't run this part!

# encode object (categorical) columns in df
enc_df = train_df1.select_dtypes(include=['object']).apply(LabelEncoder().fit_transform)

# add encoded columns back into original train df
train_df1[enc_df.columns] = enc_df

train_df1.head()

## Exploratory Data Analysis (EDA)

- Correlation 

In [ ]:
corr_matrix = train_df.corr()
corr_matrix['SalePrice'] = round(corr_matrix['SalePrice'],4)
corr_matrix['SalePrice'].sort_values(ascending=False)

- Variance Inflation Factor

In [ ]:
# not finished yet!!
# X = add_constant(train_df)
# pd.Series([variance_inflation_factor(X.values, i)
#           for i in range(X.shape[1])], index=X.columns)

## Data Exploration:

In [ ]:
# check distribution of sales price
train_df1.hist(column='SalePrice')

In [ ]:
# normalize sale price so that it can be evenly distributed
train_df1['LogPrice'] = np.log(train_df1.SalePrice)
train_df1.hist(column='LogPrice')

In [ ]:
# check for more skewed columns
train_df1.skew().sort_values(ascending=False).head(30)

In [ ]:
# looking at correlation of numeric features to SalePrice column. 
# this will suggest which columns have a greater relationship with the SalePrice column 
corr = train_df1.corr().abs().unstack().sort_values(ascending=False)['LogPrice']
corr = corr.iloc[1:]
corr = pd.DataFrame(corr, columns = ['Correlation'])

corr = corr[corr.Correlation > 0.50]
corr

In [ ]:
cols = []
for ind in corr.index:
    cols.append(ind)

# create pair plot between columns that have correlation 50% and above
sns.pairplot(train_df1[cols])

## Data Visualization (CAN BE DELETED):

In [ ]:
#Count the number of houses sold in a year
#year_sold_pivot = train_df.pivot_table(index='YrSold', values='SalePrice', aggfunc='count')
#print(year_sold_pivot)

# Plotting the sum of sales per year
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='YrSold', y='SalePrice', data=train_df[['SalePrice', 'YrSold']], estimator=sum)
plt.xlabel('Year Sold')
plt.ylabel('Number of Sales')
plt.show()

In [ ]:
# Plotting the median sale price for each year

#Checking the median price
#year_sold_pivot = train_df.pivot_table(index='YrSold', values='SalePrice', aggfunc=np.median)
#print(year_sold_pivot)

sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='YrSold', y='SalePrice', data=train_df[['SalePrice', 'YrSold']], estimator=np.median)
plt.xlabel('Year Sold')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
#Displaying the median price for each year using box plot.  
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.boxplot(x='YrSold', y='SalePrice', data=train_df[['SalePrice', 'YrSold']])
plt.xlabel('Year Sold')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
# Displaying the median price based on overall quality of the house using box plot.
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(12,10))
sns.boxplot(x='OverallQual', y='SalePrice', data=train_df[['SalePrice', 'OverallQual']])
plt.xlabel('Overall Quality')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
sns.set_context('talk', font_scale=1) 
sns.set_style('dark')
plt.figure(figsize=(15,8))

# Plot GarageArea vs sale price of house considering the number of cars that can
# fit in the garage
sns.scatterplot(x='GarageArea', y='SalePrice', hue=train_df.GarageCars.tolist(),
            palette='Set2', data=train_df[['SalePrice', 'GarageArea','GarageCars']])
plt.show()

In [ ]:
# Plotting the median sale price based on external quality
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='ExterQual', y='SalePrice', data=train_df[['SalePrice', 'ExterQual']], estimator=np.median)
plt.xlabel('External Quality')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
# Plotting the median sale price based on basement quality
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='BsmtQual', y='SalePrice', data=train_df[['SalePrice', 'BsmtQual']], estimator=np.median)
plt.xlabel('Basement Quality')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
# Plotting the median sale price based on heating and air conditioning quality
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='HeatingQC', y='SalePrice', data=train_df[['SalePrice', 'HeatingQC']], estimator=np.median)
plt.xlabel('Heating Quality')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
# Plotting the median sale price based on kitchen quality
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='KitchenQual', y='SalePrice', data=train_df[['SalePrice', 'KitchenQual']], estimator=np.median)
plt.xlabel('Kitchen Quality')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
# Plotting the median sale price based on fireplace quality
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='FireplaceQu', y='SalePrice', data=train_df[['SalePrice', 'FireplaceQu']], estimator=np.median)
plt.xlabel('Fireplace Quality')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
# Plotting the median sale price based on garage quality
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='GarageQual', y='SalePrice', data=train_df[['SalePrice', 'GarageQual']], estimator=np.median)
plt.xlabel('Garage Quality')
plt.ylabel('Median Sale Price')
plt.show()

In [ ]:
# Plotting the median sale price based on pool quality
sns.set_context('talk', font_scale=1) 
plt.figure(figsize=(10,5))
sns.barplot(x='PoolQC', y='SalePrice', data=train_df[['SalePrice', 'PoolQC']], estimator=np.median)
plt.xlabel('Pool Quality')
plt.ylabel('Median Sale Price')
plt.show()

## Feature Engineering:

In [ ]:
# sum of quality points.
train_df['SumQuality'] = train_df.ExterQual_Num + train_df.BsmtQual_Num + train_df.HeatingQC_Num + train_df.KitchenQual_Num + train_df.GarageQual_Num + train_df.FireplaceQu_Num + train_df.GarageQual_Num + train_df.PoolQC_Num 
            
#print(train_df['SumQuality'])

# sum of quality points, removing less correlated features (< 0.5).
train_df['SumMIQ'] = train_df.ExterQual_Num + train_df.BsmtQual_Num + train_df.KitchenQual_Num + train_df.FireplaceQu_Num + train_df.GarageQual_Num


In [ ]:
# Garage area per car
# May not need this one, the correlation to SalePrice is only slightly higher than GarageArea
train_df['GarageAreaPerCar'] = train_df.GarageArea + train_df.GarageCars 

#print(train_df['GarageAreaPerCar'])

In [ ]:
corr = train_df.corr().abs().unstack().sort_values(ascending=False)['SalePrice']
corr.head(40)

In [ ]:
################## DISREGARD LOGIC BELOW ######################

In [ ]:
# encode object columns
# enc_df = train_df.select_dtypes(include=['object']).apply(LabelEncoder().fit_transform)
# enc_df

In [ ]:
# add encoded columns back into train df
# train_df[enc_df.columns] = enc_df
# train_df

In [ ]:
# normalize df except for price column
# norm_df = (train_df - train_df.mean()) / (train_df.max() - train_df.min())